In [27]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers, models, Input  

from sklearn.metrics import confusion_matrix


In [28]:
# =========================
# 1. load data & Dataset define
# =========================
data_dir = r".\archive"
train_path = os.path.join(data_dir, "mitbih_train.csv")
test_path = os.path.join(data_dir, "mitbih_test.csv")

train_df = pd.read_csv(train_path, header=None)
test_df = pd.read_csv(test_path, header=None)

X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values.astype(int)

X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values.astype(int)

# Z-score per sample
X_train = (X_train - X_train.mean(axis=1, keepdims=True)) / (X_train.std(axis=1, keepdims=True) + 1e-8)
X_test = (X_test - X_test.mean(axis=1, keepdims=True)) / (X_test.std(axis=1, keepdims=True) + 1e-8)

# reshape: (samples, 187, 1), tf dif
X_train = X_train[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]


X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)

X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.int32)


In [33]:
#Defind model, couple CNN,
def ECG_CNN_TF(input_shape=(187, 1), num_classes=5):
    model = models.Sequential()
    
    # Input layer
    model.add(Input(shape=input_shape))
    
    # Conv Block 1
    model.add(layers.Conv1D(64, kernel_size=5, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv1D(64, kernel_size=5, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.MaxPooling1D(pool_size=2, strides=2))
    
    # Conv Block 2
    model.add(layers.Conv1D(128, kernel_size=5, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv1D(128, kernel_size=5, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.MaxPooling1D(pool_size=2, strides=2))
    
    # Flatten
    model.add(layers.Flatten())
    
    # FC layers
    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))
    
    # Output
    model.add(layers.Dense(num_classes))
    
    return model


In [34]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
else:
    print("Using CPU")
save_dir = os.path.join(data_dir, "models")
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, "ECG_couple_TF.keras")


Using CPU


In [38]:
# =========================
# 3.Training
# =========================

#keep same seed for all model
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

#Stratified split
class_ranges = [
    (0, 72471),
    (72471, 74694),
    (74694, 80483),
    (80483, 81123),
    (81123, 87554)
]

train_indices = []
val_indices = []

for start, end in class_ranges:
    idx = np.arange(start, end)
    np.random.shuffle(idx)
    n_val = int(len(idx) * 0.2)
    val_indices.extend(idx[:n_val])
    train_indices.extend(idx[n_val:])

X_train_split = X_train[train_indices]
y_train_split = y_train[train_indices]
X_val_split = X_train[val_indices]
y_val_split = y_train[val_indices]

#Dataset + batch
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_split, y_train_split))
train_dataset = train_dataset.shuffle(len(X_train_split)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_split, y_val_split))
val_dataset = val_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test_tf, y_test_tf))
val_dataset = val_dataset.batch(batch_size)

model = ECG_CNN_TF(input_shape=(187,1), num_classes=5)
# Loss function 
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Optimizer AdamW
optimizer = tf.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-3)

# Compile model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
save_path = model_path

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
]

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks
)




Epoch 1/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 74s 64ms/step - accuracy: 0.9395 - loss: 0.2143 - val_accuracy: 0.9709 - val_loss: 0.1052
Epoch 2/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 78s 71ms/step - accuracy: 0.9677 - loss: 0.1148 - val_accuracy: 0.9702 - val_loss: 0.0980
Epoch 3/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 76s 69ms/step - accuracy: 0.9742 - loss: 0.0907 - val_accuracy: 0.9788 - val_loss: 0.0753
Epoch 4/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 73s 67ms/step - accuracy: 0.9776 - loss: 0.0774 - val_accuracy: 0.9795 - val_loss: 0.0790
Epoch 5/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 76s 69ms/step - accuracy: 0.9804 - loss: 0.0689 - val_accuracy: 0.9788 - val_loss: 0.0820
Epoch 6/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 76s 69ms/step - accuracy: 0.9820 - loss: 0.0617 - val_accuracy: 0.9832 - val_loss: 0.0620
Epoch 7/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 81s 74ms/step - accuracy: 0.9828 - loss: 0.0584 - val_accuracy: 0.9805 - val_loss: 0.0670
Epoch 8/100
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 78s 71ms/step - accuracy: 

In [39]:
# =========================
# evaluation
# =========================
batch_size = 128

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)

# load model
model = tf.keras.models.load_model(model_path)
preds_prob = model.predict(test_dataset)  # shape: (num_samples, num_classes)
#predict 
preds = np.argmax(preds_prob, axis=1)     
#save CSV
csv_path = os.path.join(save_dir, "test_pred.csv")
pd.DataFrame({"y_true": y_test, "y_pred": preds}).to_csv(csv_path, index=False)
print(f"Evaluation CSV saved at {csv_path}")


172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step
Evaluation CSV saved at .\archive\models\test_pred.csv


In [40]:
# =========================
# confusion matrix & metrics
# =========================
cm = confusion_matrix(y_test, preds, labels=list(range(5)))

print("===== Confusion Matrix =====")
print(cm)

# calculate metrics for 5 classes
metrics_per_class = {"recall": [], "specificity": [], "precision": [], "f1": []}
class_counts = cm.sum(axis=1)
total_samples = class_counts.sum()
weights = class_counts / total_samples

for i in range(5):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)

    recall_i = TP / (TP + FN + 1e-8)
    specificity_i = TN / (TN + FP + 1e-8)
    precision_i = TP / (TP + FP + 1e-8)
    f1_i = 2 * recall_i * precision_i / (recall_i + precision_i + 1e-8)

    metrics_per_class["recall"].append(recall_i)
    metrics_per_class["specificity"].append(specificity_i)
    metrics_per_class["precision"].append(precision_i)
    metrics_per_class["f1"].append(f1_i)

macro_avg_metrics = {k: np.mean(v) for k, v in metrics_per_class.items()}
weighted_avg_metrics = {k: np.sum(np.array(v) * weights) for k, v in metrics_per_class.items()}

print("\n===== Per-Class Metrics =====")
for k, v in metrics_per_class.items():
    print(f"{k}: {np.round(v, 4)}")
print("\n===== Macro-Average Metrics =====")
for k, v in macro_avg_metrics.items():
    print(f"{k}: {v:.4f}")
print("\n===== Weighted-Average Metrics =====")
for k, v in weighted_avg_metrics.items():
    print(f"{k}: {v:.4f}")

===== Confusion Matrix =====
[[18047    49    18     2     2]
 [   91   454    11     0     0]
 [   44     3  1383    18     0]
 [   22     0    13   127     0]
 [   16     0     1     0  1591]]

===== Per-Class Metrics =====
recall: [0.9961 0.8165 0.9551 0.784  0.9894]
specificity: [0.9542 0.9976 0.9979 0.9991 0.9999]
precision: [0.9905 0.8972 0.9698 0.8639 0.9987]
f1: [0.9933 0.855  0.9624 0.822  0.9941]

===== Macro-Average Metrics =====
recall: 0.9082
specificity: 0.9897
precision: 0.9441
f1: 0.9254

===== Weighted-Average Metrics =====
recall: 0.9868
specificity: 0.9618
precision: 0.9864
f1: 0.9865
